In [1]:
%run functions.ipynb

Ready: 2023-04-24 23:45:06


In [290]:
def make_hse_image(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=28,
                     header_color='#249FD8', row_colors=['#D2DDF1', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, show=False, **kwargs):
    rating_col = df.columns[-1]
    data = df.sort_values(rating_col, ascending=False)
    data_separeted_numbers = data.copy()
    for col in data.columns[1:]:
        data_separeted_numbers[col] = data[col].apply(make_sep)
        if col in ['Пользователи, %', 'Запросы, %', 'Запросов на пользователя,\nитого']:
            data_separeted_numbers[col] = data[col].apply(lambda x: str(x).replace('.',','))
        
    if ax is None:
        size = (np.array([6.3*4, 3.94*4]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data_separeted_numbers.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    #ax.figure.figimage(im, 43, 44, zorder=1, alpha=0.5)
#
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    #ax.figure.figimage(im, 43, 1068, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 15)] + [color_list_full[-1]]) 
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.05)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index ==0 else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        elif column_index == 6:
            cell.set_width(0.0008)
        else:
            cell.set_width(0.0016)
                        
        # Цветвоая шкала
        if row_index > 0 and column_index not in cols_without_color and row_index<16:
            cell.set_facecolor(colors.pop(0).upper())       
            
        if row_index == 16:
            cell.set_text_props(color='w')
            cell.set_facecolor(header_color) 
            if column_index == 6:
                cell.set_text_props(color=header_color)
            
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()    


In [291]:
df = pd.read_excel('files/ten_city/ratings_df.xlsx').fillna(0).astype(
    {'Рубрика':str, 
'Пользователи 2020':int,
'Пользователи 2021':int,
'Пользователи 2022':int,
'Пользователи 2023':int,
'Пользователи, итого':int,
'Пользователи, %' : float,
'Запросы 2020':int,
'Запросы 2021':int,
'Запросы 2022':int,
'Запросы 2023':int,
'Запросы, итого':int,
'Запросы, %' : float,                                                                      
})

df['Запросы, %'] = df['Запросы, %'].round(1)
df['Пользователи, %'] = df['Пользователи, %'].round(1)

df_users = df[[
    'Рубрика',
    'Пользователи 2020',
    'Пользователи 2021',
    'Пользователи 2022',
    'Пользователи 2023',
    'Пользователи, итого',
    'Пользователи, %'
]].iloc[list(range(15))+[-1]].sort_values('Пользователи, итого', ascending=False)

df_requests = df[[
    'Рубрика',
    'Запросы 2020',
    'Запросы 2021',
    'Запросы 2022',
    'Запросы 2023',
    'Запросы, итого',
    'Запросы, %'
]].iloc[list(range(15))+[-1]].sort_values('Запросы, итого', ascending=False)

#249FD8
#7FA2D3
#D2DDF1

In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
make_hse_image(df_requests, f'requests_{now}.png', show=True)

In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
make_hse_image(df_users, f'users_{now}.png', show=True)

In [299]:
def make_hse_image2(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=28,
                     header_color='#249FD8', row_colors=['#D2DDF1', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, show=False, **kwargs):
    rating_col = df.columns[-1]
    data = df
    data['Население'] = data['Население'].apply(make_sep)
    if ax is None:
        size = (np.array([6.3*4, 3.94*4]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    #ax.figure.figimage(im, 43, 44, zorder=1, alpha=0.5)
#
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    #ax.figure.figimage(im, 43, 1068, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    #colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    #colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 20)] + [color_list_full[-1]]) 
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.04)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index in (0,1,2) else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.0001)
            cell.set_text_props(ha='right')
        else:
            cell.set_width(0.0006)
            
        if row_index == 0 and column_index == 0:
            cell.set_text_props(ha='center')

        # Цветвоая шкала
        if row_index > 0 and column_index == 3:
            cell.set_facecolor(colors.pop(0).upper())       
            
        if row_index == 18:
            cell.set_text_props(color='w')
            cell.set_facecolor(header_color) 
            if column_index == 1 or column_index == 2:
                cell.set_text_props(color=header_color)
            
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()    


In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
dfn = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\ВШЭ\Таблица_Население_230425_0030.xlsx")
make_hse_image2(dfn, f'table_{now}.png', show=True)